# 1. 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from datetime import datetime as dt
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

train = pd.read_csv( 'input/funda_train.csv' )
submit = pd.read_csv( 'input/submission.csv' )


# 데이터형태 확인
train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
store_id            int64
card_id             int64
card_company        object
transacted_date     object
transacted_time     object
installment_term    int64
region              object
type_of_business    object
amount              float64
dtypes: float64(1), int64(3), object(5)
memory usage: 450.2+ MB


## 1.1 전역에서 활용할 함수들을 필요할 때 마다 정의

In [3]:
# DataFrame 의 컬럼에서 max 값 확인
def get_max( df, column_key ) :
    return df[column_key][df[column_key].idxmax()]

def get_min( df, column_key ) :
    return df[column_key][df[column_key].idxmin()]

## 1.2 데이터 확인
### 1.2.1 유니크 값 확인

In [4]:
# column 별 유니크 값 개수 확인
for key in train.columns :
    print( '{}: {}'.format( key, len( train[key].unique() ) ) )

store_id: 1967
card_id: 3950001
card_company: 8
transacted_date: 1003
transacted_time: 1440
installment_term: 34
region: 181
type_of_business: 146
amount: 30551


<b>[의미있는 값]</b><br>
1. 상점은 1967개다
 - 상점별 매출액을 종합할 수 있다.
2. 카드사는 총 8개다.
 - 분별력있는 판단은 줄 수 없지만 구분값으론 의미가 있을 수도 있다.
3. 181개 지역의 매출이다
 - 지역별 매출액에 의미를 줄 수있는지 확인해볼 수 있다.
4. 업종은 146개다
 - 계절에 따른 업종별 매출액이 의미를 줄 수 있는지 확인해볼 수 있다.

### 1.2.2 매출액 확인

In [5]:
# 매출액 최대 최소값 확인
print( get_min( train, 'amount' ) )
print( get_max( train, 'amount' ) )

-2771428.5714285714
5571428.571428572


In [6]:
# 음수인 매출액 확인
train[train['amount']<0]

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
41,0,40,a,2016-06-10,17:26,2,NaN,기타 미용업,-8571.428571
347,0,285,a,2016-08-04,17:52,0,NaN,기타 미용업,-1857.142857
731,0,473,g,2016-10-17,10:32,0,NaN,기타 미용업,-2000.000000
831,0,230,b,2016-11-03,15:36,0,NaN,기타 미용업,-85.714286
944,0,138,a,2016-11-28,13:21,0,NaN,기타 미용업,-57.142857
1953,0,829,a,2017-07-10,17:19,0,NaN,기타 미용업,-18285.714286
2659,0,1182,e,2017-12-22,19:22,0,NaN,기타 미용업,-7142.857143
2734,0,1080,g,2018-01-10,17:27,0,NaN,기타 미용업,-18.571429
3256,0,626,g,2018-05-13,18:16,0,NaN,기타 미용업,-2000.000000
3544,0,1401,a,2018-07-16,14:22,0,NaN,기타 미용업,-5000.000000


<b>[확인 된 내용]</b><br>
음수값은 매출이 아니라 취소금액이므로 row 에서 드랍해야 한다.

### 1.2.3 Null Data 확인

In [7]:
# 데이터 형태를 출력해 column, row 개수 확인
train.shape

(6556613, 9)

In [8]:
# 데이터 타입 확인 (불필요하게 큰 데이터는 사이즈를 줄여줄 수 있도록)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
store_id            int64
card_id             int64
card_company        object
transacted_date     object
transacted_time     object
installment_term    int64
region              object
type_of_business    object
amount              float64
dtypes: float64(1), int64(3), object(5)
memory usage: 450.2+ MB


In [9]:
# 빈 데이터 개수 확인
train.isnull().sum() # 지역과 비즈니스 타입은 null 데이터가 많아 분석에 도움이 안 될 것으로 판단되므로 나중에 drop 을 고민하도록 한다.

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region              2042766
type_of_business    3952609
amount                    0
dtype: int64

In [10]:
# 지역 컬럼 결착율
2042766/6556613*100

31.155811697289437

In [11]:
# 업종 컬럼 결착율
3952609/6556613*100

60.28431142725672

### 1.3 확인한 값들을 이용한 처리방안
1. 데이터 개수는 6556613 개다 (매출액 별 기록이기 때문에 매장별 매출로 변환이 필요하다)
2. 매출액('amount')에 음수가 있다. (취소 처리 내역)
 - 매출액이 아니므로 데이터에서 드랍한다.
3. transacted_date, transacted_time 컬럼을 합쳐 시계열 데이터 처리를 지원해줄 수 있다.
4. 지역('region')과 업종 구분('type_of_business')은 결착율이 각가 31% 60% 이므로 변별력이 떨어진다. (드랍한다)
5. 주식지수 예측과 비슷한 단순 이동 평균 모델을 사용해 보도록 한다.
 - 월별 매출액 산출이 필요하다

# 2. 전처리

## 2.1 메모리 및 연산속도 향상을 위한 데이터 변형
1. 컬럼별 최소 최대값 확인 후 데이터 타입 변경 (메모리 사용량 최적화)

In [12]:
# int 타입인 컬럼만 최소 최대값 출력
for column_key in train.columns : # 컬럼 이름 조회
    if True == np.issubdtype( train[column_key].dtypes, np.integer ) : # 컬럼 별 데이터 타입 조회
        print( 'key: {}\n\tmin: {}\n\tmax: {}'.format( column_key, get_min(train, column_key), get_max(train, column_key) ))

key: store_id
	min: 0
	max: 2136
key: card_id
	min: 0
	max: 4663856
key: installment_term
	min: 0
	max: 93


<b>[데이터 변형을 위한 분석 결과]</b><br>
1. 컬럼 중 'store_id', 'card_id', 'installment_term' 만 정수형 변수이며 int32 최대값 (2,147,483,647) 을 넘지 않기 때문에 모든 데이터 크기를 축소해준다.<br>

In [13]:
train = train.astype( {'store_id': 'int16', 'card_id': 'int32', 'installment_term': 'int16'} )
train

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
5,0,5,c,2016-06-02,13:09,0,NaN,기타 미용업,2000.000000
6,0,6,f,2016-06-02,15:33,0,NaN,기타 미용업,2000.000000
7,0,7,a,2016-06-02,17:18,0,NaN,기타 미용업,7857.142857
8,0,8,c,2016-06-02,18:30,0,NaN,기타 미용업,2000.000000
9,0,9,a,2016-06-02,19:56,0,NaN,기타 미용업,1857.142857


## 2.2 데이터 드랍
### 2.2.1 결착율 높은 데이터 드랍

In [14]:
# 'region'과 'type_of_business' 드랍
train = train.drop( 'region', axis=1)
train = train.drop( 'type_of_business', axis=1 )
train

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,amount
0,0,0,b,2016-06-01,13:13,0,1857.142857
1,0,1,h,2016-06-01,18:12,0,857.142857
2,0,2,c,2016-06-01,18:52,0,2000.000000
3,0,3,a,2016-06-01,20:22,0,7857.142857
4,0,4,c,2016-06-02,11:06,0,2000.000000
5,0,5,c,2016-06-02,13:09,0,2000.000000
6,0,6,f,2016-06-02,15:33,0,2000.000000
7,0,7,a,2016-06-02,17:18,0,7857.142857
8,0,8,c,2016-06-02,18:30,0,2000.000000
9,0,9,a,2016-06-02,19:56,0,1857.142857


### 2.2.2 취소 데이터(매출액이 음수인 값) 드랍

In [15]:
train.drop(train.loc[train['amount']<0].index, inplace=True)
train

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,amount
0,0,0,b,2016-06-01,13:13,0,1857.142857
1,0,1,h,2016-06-01,18:12,0,857.142857
2,0,2,c,2016-06-01,18:52,0,2000.000000
3,0,3,a,2016-06-01,20:22,0,7857.142857
4,0,4,c,2016-06-02,11:06,0,2000.000000
5,0,5,c,2016-06-02,13:09,0,2000.000000
6,0,6,f,2016-06-02,15:33,0,2000.000000
7,0,7,a,2016-06-02,17:18,0,7857.142857
8,0,8,c,2016-06-02,18:30,0,2000.000000
9,0,9,a,2016-06-02,19:56,0,1857.142857


### 2.2.3 시계열 데이터 (날짜와 시간)을 합쳐준다.

In [16]:
# 날짜 합치기
train['date'] = train['transacted_date']+' '+train['transacted_time']
train['date'] = pd.to_datetime(train['date'])

train.head(5)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,amount,date
0,0,0,b,2016-06-01,13:13,0,1857.142857,2016-06-01 13:13:00
1,0,1,h,2016-06-01,18:12,0,857.142857,2016-06-01 18:12:00
2,0,2,c,2016-06-01,18:52,0,2000.000000,2016-06-01 18:52:00
3,0,3,a,2016-06-01,20:22,0,7857.142857,2016-06-01 20:22:00
4,0,4,c,2016-06-02,11:06,0,2000.000000,2016-06-02 11:06:00


In [17]:
# transacted_date 와 transacted_time 을 드랍한다.
train = train.drop( 'transacted_date', axis=1)
train = train.drop( 'transacted_time', axis=1 )
train

,store_id,card_id,card_company,installment_term,amount,date
0,0,0,b,0,1857.142857,2016-06-01 13:13:00
1,0,1,h,0,857.142857,2016-06-01 18:12:00
2,0,2,c,0,2000.000000,2016-06-01 18:52:00
3,0,3,a,0,7857.142857,2016-06-01 20:22:00
4,0,4,c,0,2000.000000,2016-06-02 11:06:00
5,0,5,c,0,2000.000000,2016-06-02 13:09:00
6,0,6,f,0,2000.000000,2016-06-02 15:33:00
7,0,7,a,0,7857.142857,2016-06-02 17:18:00
8,0,8,c,0,2000.000000,2016-06-02 18:30:00
9,0,9,a,0,1857.142857,2016-06-02 19:56:00


In [18]:
# 시간별 매출로 정렬하기 위해 date 를 index 로 지정한다.
train.set_index('date', inplace=True)

train

,store_id,card_id,card_company,installment_term,amount
date,,,,,
2016-06-01 13:13:00,0,0,b,0,1857.142857
2016-06-01 18:12:00,0,1,h,0,857.142857
2016-06-01 18:52:00,0,2,c,0,2000.000000
2016-06-01 20:22:00,0,3,a,0,7857.142857
2016-06-02 11:06:00,0,4,c,0,2000.000000
2016-06-02 13:09:00,0,5,c,0,2000.000000
2016-06-02 15:33:00,0,6,f,0,2000.000000
2016-06-02 17:18:00,0,7,a,0,7857.142857
2016-06-02 18:30:00,0,8,c,0,2000.000000


In [19]:
train.index

DatetimeIndex(['2016-06-01 13:13:00', '2016-06-01 18:12:00',
               '2016-06-01 18:52:00', '2016-06-01 20:22:00',
               '2016-06-02 11:06:00', '2016-06-02 13:09:00',
               '2016-06-02 15:33:00', '2016-06-02 17:18:00',
               '2016-06-02 18:30:00', '2016-06-02 19:56:00',
               ...
               '2019-02-28 22:31:00', '2019-02-28 22:35:00',
               '2019-02-28 22:43:00', '2019-02-28 22:45:00',
               '2019-02-28 23:03:00', '2019-02-28 23:20:00',
               '2019-02-28 23:24:00', '2019-02-28 23:24:00',
               '2019-02-28 23:27:00', '2019-02-28 23:54:00'],
              dtype='datetime64[ns]', name='date', length=6483513, freq=None)

### 2.2.4 월 매출액으로 변환

In [20]:
df_month = pd.DataFrame()
for i in tqdm( train.store_id.unique() ): #매장별 매출액 합산
    data_num = train[train.store_id==i]
    sum_amount = data_num['amount'].resample(rule='m').sum()
    data_mon = pd.concat([sum_amount],axis=1)
    data_mon.insert(0,'store_id',i)
    df_month= pd.concat([df_month,data_mon],axis=0)

df_month

100%|██████████| 1967/1967 [00:25<00:00, 76.63it/s]


,store_id,amount
date,,
2016-06-30,0,7.555714e+05
2016-07-31,0,1.005000e+06
2016-08-31,0,8.734286e+05
2016-09-30,0,8.978571e+05
2016-10-31,0,8.374286e+05
2016-11-30,0,6.971429e+05
2016-12-31,0,7.618571e+05
2017-01-31,0,5.856429e+05
2017-02-28,0,7.940000e+05


# 3. 모델링 및 예측

In [21]:
# 예측 모델을 아직 직접 구현하긴 어려우니 참조하기로 한다.
# 단 코드 분석은 해보도록 한다.
# https://www.dacon.io/competitions/official/140472/codeshare/1734?page=1&dtype=recent&ptype=pub
def sub(df,n):
    # 3달치 empty DataFrame 생성
    concat_3mon = pd.DataFrame(index=pd.to_datetime(['2019-03-31', '2019-04-30', '2019-05-31']))
    df_sma=pd.DataFrame(columns=['store_id','amount'])

    for i in df_month.store_id.unique():
        # store_id 별 amount가 담긴 sf_set 데임터 프레임을 생성
        df_set = pd.DataFrame(df[df.store_id == i].amount)
        # store_id 별 df_set에 contact_3mon 을 병합한다.
        sma_train = pd.concat([df_set,concat_3mon], axis=0)
        num = df_set.amount.rolling(window=n,min_periods=1).mean().values[-3:].sum()
        df_sma.loc[i,'store_id']=i
        df_sma.loc[i,'amount']=num
        
    return df_sma

In [22]:
#월별 예측 매출액 저장
for i in tqdm(range(4, 7)):
    sma_sub = sub(df_month, i)
    sma_sub.to_csv(f'data/funda_{i}sma_sub.csv',index=False)

100%|██████████| 3/3 [00:22<00:00,  7.41s/it]


In [25]:
# 저장된 값을 로드해 하나의 파일로 합쳐준다.
list_files = glob.glob('data/*')
df_all = []
for file in list_files :
    df = pd.read_csv(file)
    df_all.append(df)
    
df_combine = pd.concat( df_all, axis=0, ignore_index=True)
df_combine.to_csv('output/output.csv', index=False)